# Example 7: Binary Reactions

This example shows how KIPET can predict when a reaction starts using a step function


#### Set up KIPET

In [1]:
import kipet
kipet_model = kipet.KipetModel()

from pathlib import Path
cur_dir = Path.cwd()

#### Set up reaction model

In [3]:
r1 = kipet_model.new_reaction('reaction-1')   
 
# Add the model parameters
k1 = r1.parameter('k1', value=2.0, bounds=(0.0, 5.0))
k2 = r1.parameter('k2', value=0.2, bounds=(0.0, 2.0))

# Declare the components and give the initial values
A = r1.component('A', value=0.001, variance=1e-10)
B = r1.component('B', value=0.0, variance=1e-11)
C = r1.component('C', value=0.0, variance=1e-8)

# Use this function to replace the old filename set-up
filename = filename = cur_dir.joinpath('data/delayed_data.csv')
full_data = kipet_model.read_data_file(filename)

data_set = full_data.iloc[::3]
r1.add_data('C_data', data=data_set)

# Use step functions to turn on the reactions
b1 = r1.step('b1', time=2, fixed=False, switch='on')
b2 = r1.step('b2', time=2.1, fixed=False, switch='on')

# Define the reactions
rA = k1*A
rB = k2*B

# Multiply the reaction rates with the step variables
rA *= b1
rB *= b2

# Define the reaction model
r1.add_ode('A', -rA )
r1.add_ode('B', rA - rB )
r1.add_ode('C', rB )

#### Define settings and solve

In [5]:
r1.settings.collocation.nfe = 60
r1.settings.collocation.ncp = 3

r1.settings.solver.linear_solver = 'ma57'
r1.settings.parameter_estimator.sim_init = True

# # Run KIPET
r1.run_opt() 

# Display the results
r1.results.show_parameters

    number of finite elements specified in apply. The larger number of finite
    elements will be used.
Ipopt 3.13.2: nlp_scaling_method=gradient-based
linear_solver=ma57


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver ma57.

Number of nonzeros in equality constraint Jacobian...:    10615
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     2189

Total number of variables............................:     2505
                     variables with only lower bounds:        0
                variables with lower and upper bounds:

#### The predicted start times are:

In [8]:
r1.results.time_step_change

b1_0    1.981601
b2_0    2.485488
dtype: float64

#### Plot results

In [6]:
r1.plot(jupyter=True)